In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner.tuners import RandomSearch
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from keras_tuner import HyperModel, Objective
from sklearn import preprocessing

# Veri yükleme
feature_list =['ACK Flag Cnt',
 'Active Max',
 'Active Mean',
 'Active Std',
 'Bwd Header Len',
 'Bwd IAT Min',
 'Bwd IAT Tot',
 'Bwd Pkt Len Max',
 'Bwd Pkt Len Mean',
 'Bwd Pkt Len Min',
 'FIN Flag Cnt',
 'Flow Byts/s',
 'Flow Duration',
 'Flow IAT Max',
 'Flow IAT Mean',
 'Flow IAT Std',
 'Fwd IAT Mean',
 'Fwd IAT Min',
 'Fwd IAT Std',
 'Fwd IAT Tot',
 'Fwd Pkt Len Max',
 'Fwd Pkt Len Std',
 'Fwd Pkts/s',
 'Idle Max',
 'Idle Mean',
 'Idle Std',
 'Init Bwd Win Byts',
 'Pkt Len Max',
 'Pkt Len Min',
 'Pkt Len Var',
 'Pkt Size Avg',
 'Protocol',
 'Src Port',
 'Subflow Bwd Byts',
 'Tot Fwd Pkts',
 'TotLen Bwd Pkts',
 'Label']
file_list={"HPO":['./small/AD-S1.csv','./small/AD-S2.csv']}

j="HPO"
min_max_scaler = preprocessing.MinMaxScaler()

# CNN

In [4]:
df=pd.read_csv(file_list[j][0],usecols=feature_list) 
X_train = df.iloc[:,0:-1]
df['Label'] = df['Label'].astype('category')
y_train=df['Label'].cat.codes  

df=pd.read_csv(file_list[j][1],usecols=feature_list) 
X_test = df.iloc[:,0:-1]
df['Label'] = df['Label'].astype('category')
y_test=df['Label'].cat.codes  



min_max_scaler = preprocessing.MinMaxScaler()
X_train = min_max_scaler.fit_transform(X_train)
X_test = min_max_scaler.fit_transform(X_test)
# Veriyi CNN modeline uygun hale getirme
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# CNN modeli oluşturma
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Conv1D(filters=hp.Int('filters', min_value=32, max_value=128, step=32),
                             kernel_size=hp.Int('kernel_size', min_value=3, max_value=5),
                             activation='relu',
                             input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Flatten())
    
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(layers.Dense(units=hp.Int(f'dense_{i}_units', min_value=32, max_value=128, step=32),
                               activation='relu'))
        model.add(layers.Dropout(rate=hp.Float(f'dropout_{i}', min_value=0.0, max_value=0.5, step=0.1)))
    
    model.add(layers.Dense(21, activation='softmax'))  # Çok sınıflı bir problem için 3 sınıf var
    
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])  # F1Score'u kullanarak ölçüm yapalım
    
    return model



# Hiperparametre araması yapma
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',#Objective("val_f1_score", direction="max"),
    max_trials=10,
    directory='CNNmy_dir',
    project_name='cnn_hyperparameter_tuning')

tuner.search(X_train, y_train,
             epochs=10,
             validation_data=(X_test, y_test))

# En iyi modeli seçme ve değerlendirme
best_model = tuner.get_best_models(num_models=1)[0]

y_pred = best_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
f1 = f1_score(y_test, y_pred_classes, average='macro')  # Weighted F1 score'u kullanıyoruz
print("Weighted F1 Score:", f1)

best_model.summary()
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]
print("En iyi modelin hiperparametreleri:")


Trial 10 Complete [00h 01m 01s]
val_accuracy: 0.7014217972755432

Best val_accuracy So Far: 0.7437598705291748
Total elapsed time: 00h 12m 37s


C:\Users\kahra\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\saving\saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


594/594 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step
Weighted F1 Score: 0.7179405180377598


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 32, 128)             │             768 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 16, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 2048)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         262,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 21)                  │           2,709 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 282,261 (1.08 MB)

 Trainable params: 282,261 (1.08 MB)

 Non-trainable params: 0 (0.00 B)

En iyi modelin hiperparametreleri:


# LSTM

In [3]:
df=pd.read_csv(file_list[j][0],usecols=feature_list) 
X_train = df.iloc[:,0:-1]
df['Label'] = df['Label'].astype('category')
y_train=df['Label'].cat.codes  


df=pd.read_csv(file_list[j][1],usecols=feature_list) 
X_test = df.iloc[:,0:-1]
df['Label'] = df['Label'].astype('category')
y_test=df['Label'].cat.codes  


X_train = min_max_scaler.fit_transform(X_train)
X_test = min_max_scaler.fit_transform(X_test)

# Veriyi LSTM modeline uygun hale getirme
X_train = np.reshape(X_train, (X_train.shape[0],  X_train.shape[1],1))
X_test = np.reshape(X_test, (X_test.shape[0],  X_test.shape[1],1))

In [4]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.LSTM(units=hp.Int('units', min_value=32, max_value=128, step=32),
                          return_sequences=True,
                          input_shape=(X_train.shape[1], X_train.shape[2])))
    
    model.add(layers.Dropout(rate=hp.Float('dropout_1', min_value=0.0, max_value=0.5, step=0.1)))
    
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(layers.LSTM(units=hp.Int(f'lstm_{i}_units', min_value=32, max_value=128, step=32),
                              return_sequences=True))
        model.add(layers.Dropout(rate=hp.Float(f'dropout_{i+2}', min_value=0.0, max_value=0.5, step=0.1)))
    
    model.add(layers.LSTM(units=hp.Int('units_last', min_value=32, max_value=128, step=32)))
    model.add(layers.Dropout(rate=hp.Float('dropout_last', min_value=0.0, max_value=0.5, step=0.1)))
    
    model.add(layers.Dense(21, activation='softmax'))  # Çok sınıflı bir problem için 3 sınıf var
    
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                  loss='sparse_categorical_crossentropy',
                  metrics=["accuracy"])  # F1Score'u kullanarak ölçüm yapalım
    
    return model


tuner = RandomSearch(
    build_model,
    objective=Objective("val_accuracy", direction="max"),
    max_trials=10,
    directory='LSTMmy_dir',
    project_name='lstm_hyperparameter_tuning')

tuner.search(X_train, y_train,
             epochs=10,
             validation_data=(X_test, y_test))

best_model = tuner.get_best_models(num_models=1)[0]

# En iyi modelin değerlendirilmesi
y_pred = best_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
f1 = f1_score(y_test, y_pred_classes, average='macro')  # Weighted F1 score'u kullanıyoruz
print("Weighted F1 Score:", f1)

best_model.summary()
print("En iyi modelin hiperparametreleri:")
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]
print(best_hyperparameters.values)

Reloading Tuner from LSTMmy_dir\lstm_hyperparameter_tuning\tuner0.json



C:\Users\kahra\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
C:\Users\kahra\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\saving\saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 30 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


594/594 ━━━━━━━━━━━━━━━━━━━━ 26s 43ms/step
Weighted F1 Score: 0.6747829809666763


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 36, 128)             │          66,560 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 36, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 36, 96)              │          86,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 36, 96)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 36, 96)              │          74,112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 36, 96)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 128)                 │         115,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 21)                  │           2,709 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 344,981 (1.32 MB)

 Trainable params: 344,981 (1.32 MB)

 Non-trainable params: 0 (0.00 B)

En iyi modelin hiperparametreleri:
{'units': 128, 'dropout_1': 0.30000000000000004, 'num_layers': 2, 'lstm_0_units': 96, 'dropout_2': 0.4, 'units_last': 128, 'dropout_last': 0.4, 'learning_rate': 0.01, 'lstm_1_units': 96, 'dropout_3': 0.30000000000000004}


# MLP

In [5]:
df=pd.read_csv(file_list[j][0],usecols=feature_list) 
X_train = df.iloc[:,0:-1]
df['Label'] = df['Label'].astype('category')
y_train=df['Label'].cat.codes  

df=pd.read_csv(file_list[j][1],usecols=feature_list) 
X_test = df.iloc[:,0:-1]
df['Label'] = df['Label'].astype('category')
y_test=df['Label'].cat.codes  

# Veriyi ANN modeline uygun hale getirme

X_train = min_max_scaler.fit_transform(X_train)
X_test = min_max_scaler.fit_transform(X_test)


In [6]:
# ANN modeli oluşturma
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Dense(units=hp.Int('units_input', min_value=32, max_value=128, step=32),
                           activation='relu',
                           input_shape=(X_train.shape[1],)))
    
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(layers.Dense(units=hp.Int(f'units_{i}', min_value=32, max_value=128, step=32),
                               activation='relu'))
        model.add(layers.Dropout(rate=hp.Float(f'dropout_{i}', min_value=0.0, max_value=0.5, step=0.1)))
    
    model.add(layers.Dense(21, activation='softmax'))  # Çok sınıflı bir problem için 3 sınıf var
    
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                  loss='sparse_categorical_crossentropy',
                  metrics=["accuracy"])  # F1Score'u kullanarak ölçüm yapalım
    
    return model



# Hiperparametre araması yapma
tuner = RandomSearch(
    build_model,
    objective=Objective("val_accuracy", direction="max"),
    max_trials=10,
    directory='ANNmy_dir',
    project_name='ann_hyperparameter_tuning')

tuner.search(X_train, y_train,
             epochs=10,
             validation_data=(X_test, y_test))

# En iyi modeli seçme ve değerlendirme
best_model = tuner.get_best_models(num_models=1)[0]

y_pred = best_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
f1 = f1_score(y_test, y_pred_classes, average='macro')  # Weighted F1 score'u kullanıyoruz
print("Weighted F1 Score:", f1)

best_model.summary()
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]
print("En iyi modelin hiperparametreleri:")
print(best_hyperparameters.values)

Trial 10 Complete [00h 00m 27s]
val_accuracy: 0.7013691663742065

Best val_accuracy So Far: 0.7313323020935059
Total elapsed time: 00h 04m 28s


C:\Users\kahra\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\saving\saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


594/594 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Weighted F1 Score: 0.7017203702459485


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 128)                 │           4,736 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 21)                  │             693 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 32,277 (126.08 KB)

 Trainable params: 32,277 (126.08 KB)

 Non-trainable params: 0 (0.00 B)

En iyi modelin hiperparametreleri:
{'units_input': 128, 'num_layers': 3, 'units_0': 128, 'dropout_0': 0.1, 'learning_rate': 0.01, 'units_1': 64, 'dropout_1': 0.1, 'units_2': 32, 'dropout_2': 0.1}
